In [23]:
from environment import lmmsabr
from importlib import reload
reload(lmmsabr)
import matplotlib.pyplot as plt
import numpy as np

In [52]:
reload(lmmsabr)
lmm = lmmsabr.LMMSABR(resolution=26, tenor=4)
lmm.sample_starting_conditions(n_curves=1,random_curves=True)
lmm.prime()
lmm.generate_episodes(n_episodes=50000)


t_max set to 7


Generating episode blocks: 100%|██████████| 50/50 [20:43<00:00, 24.86s/block]


AttributeError: Can't pickle local object 'make_swap_indexer.<locals>.indexer'

In [60]:
mat = np.ones((2,2))
mat2 = np.ones((2,2))*2
conc = np.concatenate((mat, mat2), axis=1)

conc[:,0+mat2.shape[0]-1]

array([1., 1.])